# Practica 39 - Operaciones con pandas sobre un 'json nested'

## Analizar con pandas el archivo anidado `companies.json`

### Rehacer los códigos del notebook `JSON_Limpiar.ipynb` a partir de la `Sección 2- Análisis JSON anidado con Pandas` Desde la celda de código 25.
- Añadir algunos cálculos de estadísticas descriptivas básicas (todos los que ud. quiera) sobre el dataframe final y guardar en otro dataframe dichos cálculos estádisticos. De forma opcional añada algunos 'plots' con matplotlib si quiere.

In [25]:
import json
import pandas as pd

with open('companies.json', 'r') as file:
    data_json = json.load(file)

"""
data_json es una estructura de datos de python  que representa  el contenido del archivo JSON. Esta estructura puede ser
. Lista de diccionario: Sei al archivo JSOn es una lista de objetos JSON
. Diccionario: Si el objetp JSON es un único objeto JSON o un diccionario con objetos anidados 
"""   

# Para convertirlo en un 'DataFrame' generalmente debemos normalizarlo ya que no sabemos cual es la estructura real 
# que viene de JSON

#Convertir el diccionario en una celda JSON  
data_str = json.dumps(data_json, indent=4)
print('DICCIONARIO EN UNA CELDA JSON')
print('----------------------------')
print(data_str)  

print()

# Podemos leer directamente el archivo y convertirlo a un dataFrame

df_data_json = pd.read_json('companies.json')
print('ARCHIVO companies.json CONVERTIDO EN UN DATAFRAME')
print('-------------------------------------------------')
df_data_json




DICCIONARIO EN UNA CELDA JSON
----------------------------
{
    "companies": [
        {
            "company": "Tech Solutions",
            "employees": [
                {
                    "name": "John Doe",
                    "age": 30,
                    "address": {
                        "street": "123 Main St",
                        "city": "New York",
                        "zipcode": "10001"
                    },
                    "skills": [
                        "Python",
                        "Data Analysis"
                    ],
                    "salary": 3450
                },
                {
                    "name": "Jane Smith",
                    "age": 25,
                    "address": {
                        "street": "456 Market St",
                        "city": "San Francisco",
                        "zipcode": "94105"
                    },
                    "skills": [
                        "JavaScript",
                  

,companies
0,"{'company': 'Tech Solutions', 'employees': [{'..."
1,"{'company': 'Innovative Apps', 'employees': [{..."
2,"{'company': 'Cloud Services', 'employees': [{'..."
3,"{'company': 'AI Innovations', 'employees': [{'..."
4,"{'company': 'Cyber Security Inc', 'employees':..."
5,"{'company': 'Fintech Solutions', 'employees': ..."
6,"{'company': 'Healthcare IT', 'employees': [{'n..."
7,"{'company': 'E-commerce Experts', 'employees':..."
8,"{'company': 'Game Development Co', 'employees'..."
9,"{'company': 'EdTech Innovations', 'employees':..."


In [26]:

print("ARCHIVO JSON companies.json")
print('---------------------------')
data_json

ARCHIVO JSON companies.json
---------------------------


{'companies': [{'company': 'Tech Solutions',
   'employees': [{'name': 'John Doe',
     'age': 30,
     'address': {'street': '123 Main St',
      'city': 'New York',
      'zipcode': '10001'},
     'skills': ['Python', 'Data Analysis'],
     'salary': 3450},
    {'name': 'Jane Smith',
     'age': 25,
     'address': {'street': '456 Market St',
      'city': 'San Francisco',
      'zipcode': '94105'},
     'skills': ['JavaScript', 'React'],
     'salary': 2800},
    {'name': 'Mike Johnson',
     'age': 35,
     'address': {'street': '789 Broadway',
      'city': 'Los Angeles',
      'zipcode': '90015'},
     'skills': ['Java', 'Spring'],
     'salary': 4200}]},
  {'company': 'Innovative Apps',
   'employees': [{'name': 'Alice Brown',
     'age': 28,
     'address': {'street': '101 First Ave',
      'city': 'Seattle',
      'zipcode': '98101'},
     'skills': ['Ruby', 'Rails'],
     'salary': 3100},
    {'name': 'Bob White',
     'age': 32,
     'address': {'street': '202 Second Ave',
 

Cargar el JSON anidado en un DataFrame de Pandas.  

Usaremos `pandas.json_normalize` para aplanar la estructura JSON y cargarla en un DataFrame.

In [27]:
# Esto es una via 

import pandas as pd
import json

company_data = [];

for company in data_json['companies']:
    company_name= company['company']
    for employee in company['employees']:
        employee_data={
            'Company': company_name,
            'Name': employee['name'],
            'Age': employee['age'],
            'Street': employee['address']['street'],
            'City': employee['address']['city'],
            'Zipcode': employee['address']['zipcode'],
            'Skills': ', '.join(employee['skills']),
            'Zipcode': employee['address']['zipcode'],
            'Salary': employee['salary']
            
        }
        company_data.append(employee_data)
df_companies=pd.DataFrame(company_data) 
print("APLANANDO LA ESTRUCTURA DE companies.json PARA CARGARLO EN UN DATAFRAME (variante 1)")
print('------------------------------------------------------------------------------------')
df_companies.head() 

APLANANDO LA ESTRUCTURA DE companies.json PARA CARGARLO EN UN DATAFRAME (variante 1)
------------------------------------------------------------------------------------


,Company,Name,Age,Street,City,Zipcode,Skills,Salary
0,Tech Solutions,John Doe,30,123 Main St,New York,10001,"Python, Data Analysis",3450
1,Tech Solutions,Jane Smith,25,456 Market St,San Francisco,94105,"JavaScript, React",2800
2,Tech Solutions,Mike Johnson,35,789 Broadway,Los Angeles,90015,"Java, Spring",4200
3,Innovative Apps,Alice Brown,28,101 First Ave,Seattle,98101,"Ruby, Rails",3100
4,Innovative Apps,Bob White,32,202 Second Ave,Austin,73301,"PHP, Laravel",2300


In [28]:
#Esta es otra vía
#----------------

df_companies = pd.json_normalize(data_json['companies'], record_path='employees', meta='company')
print("APLANANDO LA ESTRUCTURA DE companies.json PARA CARGARLO EN UN DATAFRAME (variante 2)")
print('------------------------------------------------------------------------------------')
df_companies.head()

APLANANDO LA ESTRUCTURA DE companies.json PARA CARGARLO EN UN DATAFRAME (variante 2)
------------------------------------------------------------------------------------


,name,age,skills,salary,address.street,address.city,address.zipcode,company
0,John Doe,30,"[Python, Data Analysis]",3450,123 Main St,New York,10001,Tech Solutions
1,Jane Smith,25,"[JavaScript, React]",2800,456 Market St,San Francisco,94105,Tech Solutions
2,Mike Johnson,35,"[Java, Spring]",4200,789 Broadway,Los Angeles,90015,Tech Solutions
3,Alice Brown,28,"[Ruby, Rails]",3100,101 First Ave,Seattle,98101,Innovative Apps
4,Bob White,32,"[PHP, Laravel]",2300,202 Second Ave,Austin,73301,Innovative Apps


In [11]:
df_companies

,name,age,skills,salary,address.street,address.city,address.zipcode,company
0,John Doe,30,"[Python, Data Analysis]",3450,123 Main St,New York,10001,Tech Solutions
1,Jane Smith,25,"[JavaScript, React]",2800,456 Market St,San Francisco,94105,Tech Solutions
2,Mike Johnson,35,"[Java, Spring]",4200,789 Broadway,Los Angeles,90015,Tech Solutions
3,Alice Brown,28,"[Ruby, Rails]",3100,101 First Ave,Seattle,98101,Innovative Apps
4,Bob White,32,"[PHP, Laravel]",2300,202 Second Ave,Austin,73301,Innovative Apps
5,Charlie Green,29,"[AWS, Azure]",3700,303 Third St,Chicago,60601,Cloud Services
6,Dana Blue,27,"[GCP, Docker]",2700,404 Fourth St,Miami,33101,Cloud Services
7,Eva Black,33,"[TensorFlow, Keras]",4500,505 Fifth Ave,Boston,02101,AI Innovations
8,Frank White,36,"[PyTorch, SciKit-Learn]",5100,606 Sixth Ave,Denver,80201,AI Innovations
9,George Brown,31,"[Network Security, Penetration Testing]",3900,707 Seventh St,Houston,77001,Cyber Security Inc


### Limpieza de datos

Tratamiento de valores perdidos

Tratemos los valores que faltan en nuestro DataFrame. En este ejemplo, no tenemos valores perdidos, pero vamos a demostrar cómo se pueden manejar.

In [47]:
print(df_employees.isna().sum()) # Devuelve un dataFrame donde cada celda contiene 'True' si el valor del DataFrame original es 'NaN'

# Fill missing values with a placeholder or drop rows/columns with missing values
# Example: Fill missing values with 'Unknown'
df.fillna('Unknown', inplace=True) #Reemplaza todos los valores faltantes por 'Unknown' porque 'inplace=True'

name               0
age                0
skills             0
salary             0
address.street     0
address.city       0
address.zipcode    0
company            0
dtype: int64


### Renombrar Columnas

In [48]:
df.rename(columns={
    'address.street': 'street',
    'address.city': 'city',
    'address.zipcode': 'zipcode'
}, inplace=True)
print(df.head())

           name  age  salary          street           city zipcode  \
0      John Doe   30    3450     123 Main St       New York   10001   
1    Jane Smith   25    2800   456 Market St  San Francisco   94105   
2  Mike Johnson   35    4200    789 Broadway    Los Angeles   90015   
3   Alice Brown   28    3100   101 First Ave        Seattle   98101   
4     Bob White   32    2300  202 Second Ave         Austin   73301   

                                           companies     skill_1  \
0  [{'company': 'Tech Solutions', 'employees': [{...      Python   
1  [{'company': 'Tech Solutions', 'employees': [{...  JavaScript   
2  [{'company': 'Tech Solutions', 'employees': [{...        Java   
3  [{'company': 'Tech Solutions', 'employees': [{...        Ruby   
4  [{'company': 'Tech Solutions', 'employees': [{...         PHP   

         skill_2  
0  Data Analysis  
1          React  
2         Spring  
3          Rails  
4        Laravel  


In [39]:
# Expandir la columna 'address' en columnas separadas
if 'address' in df_employees.columns:
    df_address = pd.json_normalize(df_employees['address'])
else:
    df_address = pd.DataFrame()

# Expandir la columna 'skills' en columnas separadas
if 'skills' in df_employees.columns:
    df_skills = df_employees['skills'].apply(pd.Series)
else:
    df_skills = pd.DataFrame()

# Combinar los DataFrames
df_final = pd.concat([df_employees.drop(columns=['address', 'skills'], errors='ignore'), df_address, df_skills], axis=1)

# Imprimir el DataFrame final
print(df_final.head())




           name  age  salary  address.street   address.city address.zipcode  \
0      John Doe   30    3450     123 Main St       New York           10001   
1    Jane Smith   25    2800   456 Market St  San Francisco           94105   
2  Mike Johnson   35    4200    789 Broadway    Los Angeles           90015   
3   Alice Brown   28    3100   101 First Ave        Seattle           98101   
4     Bob White   32    2300  202 Second Ave         Austin           73301   

           company           0              1  
0   Tech Solutions      Python  Data Analysis  
1   Tech Solutions  JavaScript          React  
2   Tech Solutions        Java         Spring  
3  Innovative Apps        Ruby          Rails  
4  Innovative Apps         PHP        Laravel  


### Ampliación de las columnas de la lista

Si tiene columnas con listas (por ejemplo, skills), puede que desee ampliarlas en filas o columnas separadas.

In [46]:
print(df_final.head())

           name  age  salary  address.street   address.city address.zipcode  \
0      John Doe   30    3450     123 Main St       New York           10001   
1    Jane Smith   25    2800   456 Market St  San Francisco           94105   
2  Mike Johnson   35    4200    789 Broadway    Los Angeles           90015   
3   Alice Brown   28    3100   101 First Ave        Seattle           98101   
4     Bob White   32    2300  202 Second Ave         Austin           73301   

           company           0              1  
0   Tech Solutions      Python  Data Analysis  
1   Tech Solutions  JavaScript          React  
2   Tech Solutions        Java         Spring  
3  Innovative Apps        Ruby          Rails  
4  Innovative Apps         PHP        Laravel  
